# ❄️ Snowflake with Key-Pair Authentication

In this quickstart you will learn build and evaluate a simple LLM app with Snowflake Cortex, and connect to Snowflake with key-pair authentication.

Note, you'll need to have an [active Snowflake account](https://signup.snowflake.com/
) to run Cortex LLM functions from Snowflake's data warehouse.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/expositional/use_cases/snowflake_keypairauth.ipynb)

In [ ]:
# !pip install trulens trulens-providers-cortex
# !conda install -c https://repo.anaconda.com/pkgs/snowflake snowflake-snowpark-python snowflake-ml-python snowflake.core

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from snowflake.snowpark import Session
import os
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

# Retrieve the private key as a string from the environment
private_key_str = os.getenv('SNOWFLAKE_PRIVATE_KEY')

# Convert the string to bytes
private_key_bytes = private_key_str.encode()

# Load the private key
private_key = serialization.load_pem_private_key(
    private_key_bytes,
    password=None,  # or provide a password if the key is encrypted
    backend=default_backend()
)

connection_params= {
  "account":  os.environ["SNOWFLAKE_ACCOUNT"],
  "user": os.environ["SNOWFLAKE_USER"],
  "private_key":private_key,
  "role": os.environ["SNOWFLAKE_ROLE"],
  "database": os.environ["SNOWFLAKE_DATABASE"],
  "schema": os.environ["SNOWFLAKE_SCHEMA"],
  "warehouse": os.environ["SNOWFLAKE_WAREHOUSE"]
}

# Create a Snowflake session
snowflake_session = Session.builder.configs(connection_params).create()

## Create simple LLM app

In [ ]:
from snowflake.cortex import Complete

from trulens.core import Tru
from trulens.core.app.custom import instrument

class LLM:
    def __init__(self, model="snowflake-arctic"):
        self.model = model
    
    @instrument
    def complete(self, prompt):
        return Complete(self.model, prompt)
    
llm = LLM()

## Set up feedback functions.

Here we'll test answer relevance and coherence.

In [ ]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.cortex import Cortex

# Initialize LiteLLM-based feedback function collection class:
provider = Cortex(model_engine="snowflake-arctic")

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

f_coherence = Feedback(
    provider.coherence_with_cot_reasons, name="coherence"
).on_output()

In [ ]:
provider.relevance_with_cot_reasons("what color is a monkey?", "abacadbra")

## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [ ]:
from trulens.core import TruCustomApp

tru_llm = TruCustomApp(
    llm,
    app_id="Arctic",
    feedbacks=[
        f_answer_relevance,
        f_context_relevance,
        f_coherence,
    ],
)

## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [ ]:
with tru_llm as recording:
    resp = llm.complete("What do you think about Donald Trump?")

In [ ]:
resp

In [ ]:
tru.get_leaderboard()

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(tru)